In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_client.projects

In [ ]:
project.requests

In [ ]:
assert len(project.events) == 1
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)
assert len(project.requests) == 1

In [ ]:
request = project.requests[0]

In [ ]:
func = request.code
func

In [ ]:
func.code

In [ ]:
func.assets

In [ ]:
asset = func.assets[0]
asset

In [ ]:
assert len(asset.data_subjects) == 1

In [ ]:
mock = asset.mock
mock

In [ ]:
assert mock.shape == (10, 22)

In [ ]:
data = asset.data
data

In [ ]:
assert data.shape == (10, 22)

In [ ]:
op = func.output_policy_type
op

In [ ]:
print(op.policy_code)

In [ ]:
users_function = func.unsafe_function

In [ ]:
real_result = users_function(trade_data=data)
real_result

#### Accept but deposited wrong result by mistake

In [ ]:
# uploaded wrong result
result = request.accept_by_depositing_result(real_result * 100)
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)

#### Override existing result

In [ ]:
# upload correct result
result = request.accept_by_depositing_result(real_result, force=True)
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)

#### Deny a request

In [ ]:
result = request.deny(reason= "The Submitted UserCode does not add differential privacy"
            "to the output.Kindly add differential privacy and resubmit the code.")
result

In [ ]:
assert isinstance(result, sy.SyftSuccess)

In [ ]:
project.requests

### Re-approve request

In [ ]:
result = request.accept_by_depositing_result(real_result, force=True)
result

In [ ]:
project.requests

In [ ]:
if node.node_type.value == "python":
    node.land()